Ce script permet d'afficher tous les trains traversant la France ayant du retard (et uniquement du retard).

# Code

## Import des Modules

In [50]:
import pandas as pd
import requests
import json
import datetime

## Obtention des trains ayant eu des perturbations sur leur trajet

Mise en varaible du token de connexion à l'API SNCF V1

In [51]:
token = 'e7b7fedd-71d0-48c6-8cc7-749e22ba8e80'

Requête à l'API SNCF afin de récupérer la liste des trains avec des perturbations

In [52]:
req = requests.get('https://api.sncf.com/v1/coverage/sncf/disruptions?since=20230302T000000&start_page=0&count=1000&',auth=(token, ''))

In [53]:
doc = json.loads(req.text)
row = len(doc['disruptions'])
print(f'Nombre de retards : {row}')

Nombre de retards : 1000


## Traitement des retards

On  ne garde que les informations sur les perturbations et le jour de ces dernières

In [54]:
df = pd.DataFrame(doc['disruptions'])
df_retard = pd.DataFrame(list(df['severity']))
df_retard['updated_at'] = df['updated_at']

On ne garde que les trains ayant eu du retard

In [55]:
df_retard = df_retard[df_retard['effect'] == 'SIGNIFICANT_DELAYS']
df_retard = df_retard[['effect','updated_at']]

On réalise une fonction de conversion d'heures afin de calculer le retard

In [56]:
def conv_heure(string):
    return int(string[0:2]) + int(string[2:4])/60

## Obtention d'inforamtions complémentaires sur ce retard

On ajoute des informations au retard, ligne par ligne :

- Les gares de départ et d'arrivée

- Le retard et sa cause

- Les heures d'arrivée prévues et réelles du train

- L'heure de départ prévue du train

In [57]:
gare_d = []
gare_a = []
cause = []
retard = []
heure_arr_prevue = []
heure_arr_reelle = []
heure_depart = []

for i in df_retard.index:
    df_mod = pd.DataFrame(list(df['impacted_objects'][i]))
    df_mod = pd.DataFrame(list(df_mod['impacted_stops'])[0])
    df_mod = df_mod[(df_mod['amended_arrival_time'].notnull())]
    df_mod = df_mod[(df_mod['base_arrival_time'].notnull())]
    
    # On calcule le retard et on ajoute une exception si il y a un changement de jour entre les heures d'arrivée prévues et réelles
    retard_val = conv_heure(df_mod.iloc[-1]['amended_arrival_time']) - conv_heure(df_mod.iloc[-1]['base_arrival_time'])

    if retard_val<0:
        retard_val = int(round((retard_val+24)*60,1))
    else:
        retard_val = int(round(retard_val*60,1))
    
    df_mod['gare'] = pd.DataFrame(list(df_mod['stop_point']))['name']

    # On renvoie "Retard non expliqué" si la SNCF ne fournit pas plus d'informations sur le retard
    if df_mod.iloc[-1]['cause'] == "":
        df_mod.iloc[-1]['cause'] = "Retard non expliqué"

    # On ajoute les informations à des listes
    gare_d.append(df_mod.iloc[0]['gare'])
    gare_a.append(df_mod.iloc[-1]['gare'])
    cause.append(df_mod.iloc[-1]['cause'])
    retard.append(retard_val)
    heure_arr_prevue.append(df_mod.iloc[-1]['base_arrival_time'])
    heure_arr_reelle.append(df_mod.iloc[-1]['amended_arrival_time'])
    heure_depart.append(df_mod.iloc[0]['base_departure_time'])


/var/folders/bg/vbvjjnmj0hv9wmb92ngxcq9m0000gn/T/ipykernel_1244/3411774444.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mod.iloc[-1]['cause'] = "Retard non expliqué"
/var/folders/bg/vbvjjnmj0hv9wmb92ngxcq9m0000gn/T/ipykernel_1244/3411774444.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mod.iloc[-1]['cause'] = "Retard non expliqué"
/var/folders/bg/vbvjjnmj0hv9wmb92ngxcq9m0000gn/T/ipykernel_1244/3411774444.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/inde

Création d'une fonction afin d'obtenir le jour du départ

In [58]:
def get_day(string):
    string = string[:8]
    return string[6:8]+'-'+string[4:6]+'-'+string[0:4]

On ajoute les informations au dataframe

In [59]:
df_retard = df_retard.drop(['effect'], axis=1)

df_retard['Gare (départ)'] = gare_d
df_retard['Gare (arrivée)'] = gare_a
df_retard['updated_at'] = df_retard['updated_at'].apply(get_day)
df_retard["Arrivée (réelle)"] = heure_arr_reelle
df_retard["Arrivée (prévue)"] = heure_arr_prevue
df_retard["Départ (prévu)"] = heure_depart
df_retard['Retard (min)'] = retard
df_retard['Cause'] = cause

df_retard.rename(columns = {'updated_at':'Jour'}, inplace = True)


On passe les heures au format heure de `datetime`

In [60]:
def str_tps(str):
    return datetime.datetime.strptime(str, '%H%M%S').time()

df_retard["Arrivée (réelle)"] = df_retard["Arrivée (réelle)"].apply(str_tps)
df_retard["Arrivée (prévue)"] = df_retard["Arrivée (prévue)"].apply(str_tps)
df_retard["Départ (prévu)"] = df_retard["Départ (prévu)"].apply(str_tps)

On affiche le dataframe final

In [61]:
df_retard

,Jour,Gare (départ),Gare (arrivée),Arrivée (réelle),Arrivée (prévue),Départ (prévu),Retard (min),Cause
0,04-03-2023,Bar-le-Duc,Paris Est,11:28:00,11:23:00,09:05:00,5,Difficultés lors de la préparation du train
2,04-03-2023,Toulouse Matabiau,Muret,23:06:00,23:01:00,22:42:00,5,Prise en charge de clients en correspondance
4,04-03-2023,Calais Ville,Lille Flandres,07:41:00,07:21:00,06:06:00,20,Défaillance de matériel
5,04-03-2023,Annecy,Chambéry - Challes-les-Eaux,08:19:00,08:14:00,07:19:00,5,Régulation du trafic
10,04-03-2023,Strasbourg,Saint-Louis,10:11:30,10:01:30,08:51:00,10,Régulation du trafic
...,...,...,...,...,...,...,...,...
993,04-03-2023,Strasbourg,Molsheim,08:42:00,08:12:00,07:48:00,30,Difficultés lors de la préparation du train
994,04-03-2023,Paris Austerlitz,Orléans,09:03:00,07:13:00,05:38:00,110,Régulation du trafic
995,04-03-2023,Le Mans,Caen,10:16:00,10:11:00,08:26:00,5,Prise en charge de clients en correspondance
996,04-03-2023,Clamecy,Auxerre-Saint-Gervais,09:45:00,09:40:00,08:24:00,5,Embouteillage


On exporte le dataframe en fichier _CSV_

In [63]:
df_retard.to_csv('delays.csv', sep=',', index=False, header=True)